# SAE S2 : Labyrinthe

In [3]:
from random import *

## I) Définition de la classe :

In [2]:
class Maze:
    """
    Classe Labyrinthe
    Représentation sous forme de graphe non-orienté
    dont chaque sommet est une cellule (un tuple (l,c))
    et dont la structure est représentée par un dictionnaire
      - clés : sommets
      - valeurs : ensemble des sommets voisins accessibles
    """
    def __init__(self, height, width, empty):
        """
        Constructeur d'un labyrinthe de height cellules de haut 
        et de width cellules de large 
        L'attribut empty défini si le labyrinthe est vide (True) ou plein (False)
        Les voisinages sont initialisés à des ensembles vides
        Remarque : dans le labyrinthe créé, chaque cellule est complètement emmurée
        """
        self.height    = height
        self.width     = width
        self.neighbors = {(i,j): set() for i in range(height) for j in range (width)}
        if empty:
            for i in range(height):
                for j in range(width):
                    if i > 0:
                        self.neighbors[(i,j)].add((i-1,j))
                    if i < height-1:
                        self.neighbors[(i,j)].add((i+1,j))
                    if j > 0:
                        self.neighbors[(i,j)].add((i,j-1))
                    if j < width-1:
                        self.neighbors[(i,j)].add((i,j+1))         
        
    def info(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
        Retour:
            chaîne (string): description textuelle des attributs de l'objet
        """
        txt = "**Informations sur le labyrinthe**\n"
        txt += f"- Dimensions de la grille : {self.height} x {self.width}\n"
        txt += "- Voisinages :\n"
        txt += str(self.neighbors)+"\n"
        valid = True
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            for c2 in self.neighbors[c1]:
                if c1 not in self.neighbors[c2]:
                    valid = False
                    break
            else:
                continue
            break
        txt += "- Structure cohérente\n" if valid else f"- Structure incohérente : {c1} X {c2}\n"
        return txt

    def __str__(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
        Retour:
             chaîne (str) : chaîne de caractères représentant le labyrinthe
        """
        txt = ""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += "   ┃" if (0,j+1) not in self.neighbors[(0,j)] else "    "
        txt += "   ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += "   ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else "    "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"
        return txt
    
    def get_cells(self):
        """
        Retourne toutes les cellules du labyrinthes
        Retour:
            liste (list) : liste contenant toutes les cellules d'un labyrinthe
        """
        L = []
        for i in range(self.height):
            for j in range(self.width):
                L.append((i,j))
        return L
    
    def add_wall(self, c1, c2):
        """
        Ajout d'un mur, ce qui revient à supprimer un mur du dictionnaire
        voisins de l'autres, et inversement.
        Arguments:
            c1 (tuple) : Cellule
            c2 (tuple): Voisine de c1
        """
    # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
        0 <= c1[1] < self.width and \
        0 <= c2[0] < self.height and \
        0 <= c2[1] < self.width, \
        f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
    # Ajout du mur
        if c2 in self.neighbors[c1]:      # Si c2 est dans les voisines de c1
            self.neighbors[c1].remove(c2) # on le retire
        if c1 in self.neighbors[c2]:      # Si c3 est dans les voisines de c2
            self.neighbors[c2].remove(c1) # on le retire
        
    def remove_wall(self,c1, c2):
        """
        Supprime un mur, ce qui revient donc à ajouter un mur au dictionnaire
        de son voisin et inversement.
        Arguments:
            c1 (tuple) : Cellule
            c2 (tuple): Voisine de c1
        """
        assert 0 <= c1[0] < self.height and \
        0 <= c1[1] < self.width and \
        0 <= c2[0] < self.height and \
        0 <= c2[1] < self.width, \
        f"Erreur lors de la suppression d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # suppresion du mur
        if c2 not in self.neighbors[c1]:      # Si c2 n'est pas dans les voisines de c1
            self.neighbors[c1].add(c2) # on l'ajoute
        if c1 not in self.neighbors[c2]:      # Si c1 n'est pas dans les voisines de c2
            self.neighbors[c2].add(c1) # on l'ajoute
            
    def get_walls(self):
        """
        Retourne tous les murs du labyrinthe sous forme d'une liste de tuple de coordonnées.
        """
        L = []
        cells = self.get_cells()
        for i in range(self.height):
            for j in range(self.width):
                c1 = (i,j)
                c2 = (i,j+1)
                c3 = (i+1,j)
                if c2 in cells and c2 not in self.neighbors[c1]:
                    L.append((c1,c2))
                if c3 in cells and c3 not in self.neighbors[c1]:
                    L.append((c1,c3))
        return L
    
    def fill(self):
        """
        Complète tous les murs d'un labyrinthe.
        """
        walls = self.get_walls()
        cells = self.get_cells()
        for i in range (self.height):
            for j in range(self.width):
                c1 = (i,j)
                c2 = (i,j+1)
                c3 = (i+1,j)
                if c1 in cells and c2 in cells:
                    self.add_wall(c1,c2)
                if c1 in cells and c3 in cells:
                    self.add_wall(c1,c3)
    
    def empty(self):
        """
        Supprime tous les murs d'un labyrinthe.
        """
        for i in range(self.height):
            for j in range(self.width):
                if i > 0:
                    self.neighbors[(i,j)].add((i-1,j))
                if i < self.height-1:
                    self.neighbors[(i,j)].add((i+1,j))
                if j > 0:
                    self.neighbors[(i,j)].add((i,j-1))
                if j < self.width-1:
                    self.neighbors[(i,j)].add((i,j+1))
                    
    def get_contiguous_cells(self,c):
        """
        Retourne la liste de coordonnées des cellules contigües à celle passé en paramètre.
        Arguments:
            c1 (tuple) : Cellule
        Retour:
            list (liste) : Liste des cellules contigües
        """
        L = []
        cells = self.get_cells()
        if c[0]-1 >=0:
            L.append((c[0]-1,c[1]))
        if c[0]+1 < self.height:
            L.append((c[0]+1,c[1]))
        if c[1]-1 >=0:
            L.append((c[0],c[1]-1))
        if c[1]+1 < self.height:
            L.append((c[0],c[1]+1))
        return L
    
    def get_reachable_cells(self,c):
        """
        Retourne la liste des cellules accessibles depuis la cellule passée en paramètre.
        Arguments:
            c1 (tuple) : Cellule
        Retour:
            list (liste) : Liste des cellules accessibles.
        """
        L = []
        walls = self.get_walls()
        if c[0]-1 >=0 and ((c[0]-1,c[1]),c) not in walls:
            L.append((c[0]-1,c[1]))
        if c[0]+1 < self.height and (c,(c[0]+1,c[1])) not in walls :
            L.append((c[0]+1,c[1]))
        if c[1]-1 >=0 and ((c[0],c[1]-1),c) not in walls:
            L.append((c[0],c[1]-1))
        if c[1]+1 < self.height and (c,(c[0],c[1]+1)) not in walls:
            L.append((c[0],c[1]+1))
        return L
                          
    def gen_btree(self,h,w):
        """
        Génération d'un labyrinthe à partir d'un arbre de binaire. L'arbre binaire
        sert donc de support au labyrinthe.
        Arguments :
            h (int) : Hauteur du labyrinthe
            w (int) : Largeur du labyrinthe
        Retour:
            Maze: Labyrinthe obtenu à partir d'un arbre binaire. 
        """
        self = Maze(h,w, False)
        cells = self.get_cells()
        for i in range(self.height):
            for j in range(self.width):
                c1 = (i,j)
                c2 = (i,j+1)
                c3 = (i+1,j)
                if c2 != c1 and c3 != c1 and c2 in cells and c3 in cells:
                    hasard = randint(2,3)
                    if hasard == 2:
                        self.remove_wall(c1,c2)
                    else:
                        self.remove_wall(c1,c3)
                elif c2 != c1 and c2 in cells:
                    self.remove_wall(c1,c2)
                elif c3 != c1 and c3 in cells:
                    self.remove_wall(c1,c3)
        return self
    
    def gen_sidewinder(self,height, width):
        """
        Génération d'un labyrinthe à partir de l'algorithme de Sidewinder. Il casse
        aléatoirement le mur est ou sud d'une cellule, en parcourant le labyrinthe de l'ouest
        à l'est.
        Arguments :
            height (int) : Hauteur du labyrinthe
            width (int) : Largeur du labyrinthe
        Retour:
            Maze: Labyrinthe obtenu avec cet algorithme.
        """
        self = Maze(height, width, False)
        for i in range(self.height-1):
            sequence = []
            for j in range(self.width-1):
                sequence.append((i,j))
                c2 = (i,j+1)
                if randint(0,1) == 1:
                    self.remove_wall((i,j),c2)
                else:
                    h = randint(0,len(sequence)-1)
                    self.remove_wall(sequence[h],(sequence[h][0]+1,sequence[h][1]))
                    sequence = []
            sequence.append((i,width-1))
            hasard = randint(0, len(sequence)-1)
            cell_hasard = sequence[hasard]
            self.remove_wall(cell_hasard,(cell_hasard[0]+1,cell_hasard[1]))
        for i in range(height-1):
            self.remove_wall((height-1,i),(height-1,i+1))
        return self

    def gen_fusion(self, h, w):
        """
        Génération d'un labyrinthe à partir de l'algorithe de fusion de chemin : on
        casse aléatoirement des murs pour éviter des cycles qui se répeteraient.
        Arguments :
            h (int) : Hauteur du labyrinthe
            w (int) : Largeur du labyrinthe
        Retour:
            Maze: Labyrinthe obtenu avec cet algorithme.
        """
        #initialisation
        self = Maze(h, w, False) #on remplit le labyrinthe avec tous les murs possibles
        cells = {self.get_cells()[label] : label+1 for label in range(len(self.get_cells()))} #on labélise les cellules de 1 à n
        #on extrait la liste de tous les murs et on les « mélange » (on les permute aléatoirement)
        walls = self.get_walls()
        shuffle(walls)
        # Pour chaque mur de la liste
        for mur in range(len(walls)):
            mur_1 = walls[mur][0]
            mur_2 = walls[mur][1]
            if cells[mur_1] != cells[mur_2]: #Si les deux cellules séparées par le mur n’ont pas le même label
                self.remove_wall(mur_1,mur_2) #casser le mur
                #affecter le label de l’une des deux cellules, à l’autre, et à toutes celles qui ont le même label que la deuxième
                label = cells[mur_1]
                cells[mur_1] = cells[mur_2]
                for cle,lab in cells.items():
                    if lab == label:
                        cells[cle] = cells[mur_2]
        return self
     
    def gen_exploration(self,h, w):
        """
        Génération d'un labyrinthe à partir d'une exploration du labyrinthe, en
        cassant aléatoirement des murs.
        Arguments :
            h (int) : Hauteur du labyrinthe
            w (int) : Largeur du labyrinthe
        Retour:
            Maze: Labyrinthe obtenu avec cet algorithme.
        """
        self = Maze(h,w,False)
        cells = {self.get_cells()[c] : False for c in range(len(self.get_cells()))}
        hasard = randint(0,len(cells)-1)
        cell = self.get_cells()[0]
        cells[cell] = True
        pile = [cell]
        while pile != []:
            cellule = pile.pop(-1)
            contigue = self.get_contiguous_cells(cellule)
            voisins_non = []
            for contig in contigue:
                if not cells[contig]:
                    voisins_non.append(contig)
            if contigue != [] and voisins_non != []:
                pile.append(cellule)
                hasard = randint(0,len(voisins_non)-1)
                cellule_2 = voisins_non[hasard]
                self.remove_wall(cellule,cellule_2)
                cells[cellule_2] = True
                pile.append(cellule_2)
        return self
    
    def gen_wilson(self,h,w):
        """
        Génération d'un labyrinthe à partir de l'algorithme de Wilson.
        Arguments :
            h (int) : Hauteur du labyrinthe
            w (int) : Largeur du labyrinthe
        Retour:
            Maze: Labyrinthe obtenu avec cet algorithme.
        """
        self = Maze(h,w,False)
        cells = {self.get_cells()[c] : False for c in range(len(self.get_cells()))}
        hasard = randint(0,len(cells) - 1)
        cell = self.get_cells()[hasard]
        cells[cell] = True
        while False in cells.values():
            i = True
            while i:
                hasard = randint(0,len(cells) - 1)
                if cells[self.get_cells()[hasard]] == False:
                    cellule = self.get_cells()[hasard]
                    i = False
            chemin = [cellule]
            while True:
                cs = self.get_contiguous_cells(cellule)
                hasard = randint(0,len(cs)-1)
                cellule = cs[hasard]
                if cellule in chemin:
                    chemin.clear()
                    break
                elif cells[cellule]:
                    chemin.append(cellule)
                    break
                else:
                    chemin.append(cellule)
            if chemin:
                for cel in range(len(chemin)-1):
                    cells[chemin[cel]] = True
                    self.remove_wall(chemin[cel],chemin[cel+1])
        return self
    
    def overlay(self, content=None):
        """
        Rendu en mode texte, sur la sortie standard, \
        d'un labyrinthe avec du contenu dans les cellules
        Argument:
            content (dict) : dictionnaire tq content[cell] contient le caractère à afficher au milieu de la cellule
        Retour:
            string
        """
        if content is None:
            content = {(i,j):' ' for i in range(self.height) for j in range(self.width)}
        else:
            # Python >=3.9
            #content = content | {(i, j): ' ' for i in range(
            #self.height) for j in range(self.width) if (i,j) not in content}
            # Python <3.9
            new_content = {(i, j): ' ' for i in range(self.height) for j in range(self.width) if (i,j) not in content}
            content = {**content, **new_content}
        txt = r""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += " "+content[(0,j)]+" ┃" if (0,j+1) not in self.neighbors[(0,j)] else " "+content[(0,j)]+"  "
        txt += " "+content[(0,self.width-1)]+" ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += " "+content[(i+1,j)]+" ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else " "+content[(i+1,j)]+"  "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"
        return txt
    
    def solve_dfs(self,start, stop):
        """
        Résolution d'un labyrinthe en utilisant un parcours en longueur.
        Argument:
            start (int) : Point de départ
            stop (int) : Point d'arrivée
        Retour:
            list (liste) : Liste du chemin résolu
        """
        cells = {self.get_cells()[c] : False for c in range(len(self.get_cells()))}
        pred = {}
        pile = [start]
        cells[start] = True
        pred[start] = start
        termine = False
        while False in cells.values() and not termine:
            c = pile.pop(0)
            if c == stop:
                termine = True
            else:
                for v in self.get_reachable_cells(c):
                    if not cells[v]:
                        cells[v] = True
                        pile.insert(0,v)
                        pred[v] = c
        c = stop
        chemin = []
        while c != start:
            chemin.append(c)
            c = pred[c]
        chemin.append(start)
        return chemin
    
    def solve_bfs(self,start, stop):
        """
        Résolution d'un labyrinthe en utilisant un parcours en largeur.
        Argument:
            start (int) : Point de départ
            stop (int) : Point d'arrivée
        Retour:
            list (liste) : Liste du chemin résolu
        """
        cells = {self.get_cells()[c] : False for c in range(len(self.get_cells()))}
        pred = {}
        file = [start]
        cells[start] = True
        pred[start] = start
        termine = False
        while False in cells.values() and not termine:
            c = file.pop(0)
            if c == stop:
                termine = True
            else:
                for v in self.get_reachable_cells(c):
                    if not cells[v]:
                        cells[v] = True
                        file.append(v)
                        pred[v] = c
        c = stop
        chemin = []
        while c != start:
            chemin.append(c)
            c = pred[c]
        chemin.append(start)
        return chemin
    
    def solve_rhr(self, start, stop):
        """
        Résolution d'un labyrinthe en utilisant la main droite : un parcours en longueur sans
        connaissance du labyrinthe.
        Argument:
            start (int) : Point de départ
            stop (int) : Point d'arrivée
        Retour:
            list (liste) : Liste du chemin résolu
        """
        cells = {self.get_cells()[c] : False for c in range(len(self.get_cells()))}
        pred = {}
        cells[start] = True
        file = [start]
        pred[start] = start
        termine = False
        while False in cells.values() and not termine:
            main_dr = file.pop(0)
            voisins = self.get_reachable_cells(main_dr)
            # tri des voisins
            v = 0
            if voisins[-1][0] == main_dr[0] and voisins[-1][0] == main_dr[1]+1:
                v = voisins.pop(-1)
            sorted(voisins,reverse = True,key=lambda x: x[0])
            if v != 0:
                voisins.append(v)
            # ils sont triés dans l'ordre "bas","gauche","haut","droite"
            voisin = voisins[0]
            if main_dr == stop:
                termine = True
            else:
                if len(voisins) == 1 and voisin == pred[main_dr]:
                    while len(voisins) == 1:
                        voisin = main_dr
                        main_dr = pred[main_dr]
                        voisins = self.get_reachable_cells(main_dr)
                        # tri des voisins
                        v = 0
                        if voisins[-1][0] == main_dr[0] and voisins[-1][0] == main_dr[1]+1:
                            v = voisins.pop(-1)
                        sorted(voisins,reverse = True,key=lambda x: x[0])
                        if v != 0:
                            voisins.append(v)
                        # ils sont triés dans l'ordre "bas","gauche","haut","droite"
                    voisin = voisins[1]
                i = 1
                while cells[voisin] and i < len(voisins):
                    voisin = voisins[i]
                    i+=1
                if not cells[voisin]:
                    cells[voisin] = True
                    file.append(voisin)
                    pred[voisin] = main_dr
                if cells[voisin]:
                    file.append(main_dr)
        main_dr = stop
        chemin = []
        while main_dr != start:
            chemin.append(main_dr)
            main_dr = pred[main_dr]
        chemin.append(start)
        return chemin
    
    def distance_geo(self,c1,c2):
        """
        Retourne la distance géodisque entre les deux cellules passées en paramètre.
        Arguments :
            c1 (int) : Cellule
            c2 (int) : Cellule
        Retour:
            Entier (int) : Distance géodisque entre c1 et c2. 
        """
        return len(self.solve_dfs(c1,c2))-1
        
    def distance_man(self,c1,c2):
        """
        Retourne la distance de Manhattan entre les deux cellules passées en paramètre.
        Cette distance correspond à la distance entre deux cellules si le labyrinthe
        est vide
        Arguments :
            c1 (int) : Cellule
            c2 (int) : Cellule
        Retour:
            Entier (int) : Distance de Manhattan.
        """
        longueur = c2[0]+ c2[1] - c1[0] - c1[1]
        return abs(longueur)

## II) Exemple de base :

#### a) Affichage du labyrinthe et des infos :

In [121]:
laby = Maze(4, 4)
print(laby.info())
print(laby)

TypeError: __init__() missing 1 required positional argument: 'empty'

#### b) Cassage de murs :

In [110]:
laby.neighbors = {
    (0, 0): {(1, 0)},
    (0, 1): {(0, 2), (1, 1)},
    (0, 2): {(0, 1), (0, 3)},
    (0, 3): {(0, 2), (1, 3)},
    (1, 0): {(2, 0), (0, 0)},
    (1, 1): {(0, 1), (1, 2)},
    (1, 2): {(1, 1), (2, 2)},
    (1, 3): {(2, 3), (0, 3)},
    (2, 0): {(1, 0), (2, 1), (3, 0)},
    (2, 1): {(2, 0), (2, 2)},
    (2, 2): {(1, 2), (2, 1)},
    (2, 3): {(3, 3), (1, 3)},
    (3, 0): {(3, 1), (2, 0)},
    (3, 1): {(3, 2), (3, 0)},
    (3, 2): {(3, 1)},
    (3, 3): {(2, 3)}
}
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



#### c) Ajout d'un mur / Suppression d'une arête :

In [94]:
laby.neighbors[(1,3)].remove((2,3))
laby.neighbors[(2,3)].remove((1,3))
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋━━━┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



#### d) Suppresion de ce mur :

In [95]:
laby.neighbors[(1, 3)].add((2, 3))
laby.neighbors[(2, 3)].add((1, 3))
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



#### e) Création d'une structure incohérente volontaire :

In [96]:
laby.neighbors[(1, 3)].remove((2, 3))
print(laby)
print(laby.info())

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋━━━┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛

**Informations sur le labyrinthe**
- Dimensions de la grille : 4 x 4
- Voisinages :
{(0, 0): {(1, 0)}, (0, 1): {(1, 1), (0, 2)}, (0, 2): {(0, 1), (0, 3)}, (0, 3): {(0, 2), (1, 3)}, (1, 0): {(2, 0), (0, 0)}, (1, 1): {(0, 1), (1, 2)}, (1, 2): {(1, 1), (2, 2)}, (1, 3): {(0, 3)}, (2, 0): {(1, 0), (2, 1), (3, 0)}, (2, 1): {(2, 0), (2, 2)}, (2, 2): {(1, 2), (2, 1)}, (2, 3): {(3, 3), (1, 3)}, (3, 0): {(3, 1), (2, 0)}, (3, 1): {(3, 2), (3, 0)}, (3, 2): {(3, 1)}, (3, 3): {(2, 3)}}
- Structure incohérente : (2, 3) X (1, 3)



#### f) Correction de cette structure incohérente :

In [97]:
#laby.neighbors[(2, 3)].remove((1,3))
c1 = (1, 3)
c2 = (2, 3)
if c1 in laby.neighbors[c2] and c2 in laby.neighbors[c1]:
    print(f"Il n'y a pas de mur entre {c1} et {c2} car elles sont mutuellement voisines")
elif c1 not in laby.neighbors[c2] and c2 not in laby.neighbors[c1]:
    print(f"Il y a un mur entre {c1} et {c2} car {c1} n'est pas dans le voisinage de {c2} et {c2} n'est pas dans le voisinage de {c1}")
else:
    print(f"Il y a une incohérence de réciprocité des voisinages de {c1} et {c2}")

Il y a une incohérence de réciprocité des voisinages de (1, 3) et (2, 3)


#### g) Lister l'ensemble des cellules (parcours)

In [98]:
L = []
for i in range(laby.height):
    for j in range(laby.width):
        L.append((i,j))
print(f"Liste des cellules : \n{L}")

Liste des cellules : 
[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)]


## III) Manipulation

Avant de tester les fonctions, on commence par tester get_cells, qui récupère toutes les cellules d'une grille.

In [5]:
laby = Maze(4, 4, False)
print(laby.get_cells())

[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)]


#### a) Test de add_wall

In [100]:
laby = Maze(4, 4, True)
print(laby)
laby.add_wall((0,0),(0,1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛



#### b) Test de remove_wall

In [142]:
laby = Maze(4, 4, False)
print(laby)
laby.remove_wall((0,0),(0,1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛

┏━━━┳━━━┳━━━┳━━━┓
┃       ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



#### c) Test de get_walls

In [6]:
laby = Maze(4, 4, False)
print(laby.get_walls())
print("\n")
laby = Maze(4, 4, True)
print(laby.get_walls())

[((0, 0), (0, 1)), ((0, 0), (1, 0)), ((0, 1), (0, 2)), ((0, 1), (1, 1)), ((0, 2), (0, 3)), ((0, 2), (1, 2)), ((0, 3), (1, 3)), ((1, 0), (1, 1)), ((1, 0), (2, 0)), ((1, 1), (1, 2)), ((1, 1), (2, 1)), ((1, 2), (1, 3)), ((1, 2), (2, 2)), ((1, 3), (2, 3)), ((2, 0), (2, 1)), ((2, 0), (3, 0)), ((2, 1), (2, 2)), ((2, 1), (3, 1)), ((2, 2), (2, 3)), ((2, 2), (3, 2)), ((2, 3), (3, 3)), ((3, 0), (3, 1)), ((3, 1), (3, 2)), ((3, 2), (3, 3))]


[]


#### d) Test de fill

In [7]:
laby = Maze(5,5,True)
print(laby)
laby.fill()
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



#### e) Test de empty

In [8]:
laby = Maze(4,4,False)
print(laby)
laby.empty()
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛

┏━━━┳━━━┳━━━┳━━━┓
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛



#### f) Test de get_contiguous_cells

In [9]:
laby = Maze(4,4,True)
print(laby)
laby.get_contiguous_cells((1,1))

┏━━━┳━━━┳━━━┳━━━┓
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛



[(0, 1), (2, 1), (1, 0), (1, 2)]

#### g) Test de get_reachable_cells(c)

In [11]:
laby = Maze(4,4,True)
print(laby)
laby.get_reachable_cells((2,2))

┏━━━┳━━━┳━━━┳━━━┓
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛



[(1, 2), (3, 2), (2, 1), (2, 3)]

## IV) Génération

#### a) Test de gen_btree(h, w) 

In [33]:
laby = Maze(4,4,True)
print(laby)
print(laby.gen_btree(15,15))

┏━━━┳━━━┳━━━┳━━━┓
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃       ┃                   ┃           ┃   ┃           ┃   ┃
┣━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ┫
┃               ┃       ┃   ┃           ┃           ┃   ┃   ┃
┣━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ┫
┃   ┃           ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃       ┃   ┃   ┃
┣   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋━━━╋   ╋   ╋   ┫
┃   ┃   ┃   ┃   ┃   ┃       ┃           ┃   ┃   ┃   ┃       ┃
┣   ╋   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ┫
┃   ┃           ┃       ┃           ┃   ┃   ┃               ┃
┣   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋   ┫
┃       ┃           ┃   ┃   ┃       ┃       ┃   ┃       ┃   ┃
┣━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋   ┫
┃   ┃   ┃   ┃                  

#### b) Test de gen_sidewinder(h, w)

In [32]:
laby = Maze(15,15,True)
print(laby)
print(laby.gen_sidewinder(15,15))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃                                                           ┃
┣   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ┫
┃                                                           ┃
┣   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ┫
┃                                                           ┃
┣   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ┫
┃                                                           ┃
┣   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ┫
┃                                                           ┃
┣   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ┫
┃                                                           ┃
┣   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ┫
┃                                                           ┃
┣   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ┫
┃                                                           ┃
┣   ╋   

#### c) Test de gen_fusion(h, w)

In [3]:
laby = Maze(15,15, False)
print(laby.gen_fusion(h = 15,w = 15))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃           ┃   ┃   ┃               ┃   ┃   ┃       ┃       ┃
┣━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋   ┫
┃           ┃           ┃       ┃   ┃               ┃   ┃   ┃
┣━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ┫
┃               ┃           ┃               ┃           ┃   ┃
┣━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━┫
┃           ┃   ┃   ┃           ┃           ┃   ┃           ┃
┣━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━┫
┃   ┃   ┃   ┃       ┃       ┃   ┃           ┃   ┃           ┃
┣   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━┫
┃       ┃       ┃                   ┃           ┃       ┃   ┃
┣   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ┫
┃       ┃   ┃               ┃   ┃           ┃       ┃       ┃
┣   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ┫
┃               ┃           ┃   ┃           ┃               ┃
┣   ╋━━━

#### d) Test de gen_exploration(h, w)

In [37]:
laby = Maze(15,15, False)
print(laby.gen_exploration(h = 15,w = 15))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━

#### e) Test de gen_wilson()

In [7]:
laby = Maze(15,15, False)
print(laby.gen_wilson(h = 15,w = 15))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃       ┃       ┃           ┃           ┃                   ┃
┣━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ┫
┃       ┃   ┃                       ┃       ┃   ┃       ┃   ┃
┣   ╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ┫
┃   ┃           ┃           ┃           ┃       ┃   ┃   ┃   ┃
┣━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ┫
┃   ┃   ┃   ┃       ┃   ┃       ┃   ┃   ┃       ┃   ┃   ┃   ┃
┣   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ┫
┃       ┃           ┃           ┃   ┃   ┃           ┃   ┃   ┃
┣━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ┫
┃   ┃   ┃                       ┃   ┃   ┃       ┃   ┃       ┃
┣   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ┫
┃           ┃       ┃       ┃   ┃       ┃   ┃   ┃           ┃
┣━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ┫
┃                       ┃   ┃       ┃   ┃   ┃       ┃   ┃   ┃
┣   ╋━━━

## V) Résolution

#### a) Test de overlay(content)

In [3]:
laby = Maze(4,4, empty = True)
print(laby.overlay({
    (0, 0):'c',
    (0, 1):'o',
    (1, 1):'u',
    (2, 1):'c',
    (2, 2):'o',
    (3, 2):'u',
    (3, 3):'!'}))

┏━━━┳━━━┳━━━┳━━━┓
┃ c   o         ┃
┣   ╋   ╋   ╋   ┫
┃     u         ┃
┣   ╋   ╋   ╋   ┫
┃     c   o     ┃
┣   ╋   ╋   ╋   ┫
┃         u   ! ┃
┗━━━┻━━━┻━━━┻━━━┛



#### b) Test de solve_dfs(start, stop)

In [11]:
laby = Maze(4,4, empty = True)
laby = laby.gen_fusion(15, 15)
solution = laby.solve_dfs((0, 0), (14, 14))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D   *   * ┃       ┃               ┃               ┃       ┃
┣━━━╋   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋   ┫
┃       ┃ *     ┃           ┃               ┃   ┃   ┃   ┃   ┃
┣   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━┫
┃   ┃   ┃ *   * ┃   ┃       ┃       ┃   ┃   ┃   ┃           ┃
┣   ╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ╋━━━╋━━━┫
┃   ┃   ┃     * ┃   ┃       ┃   ┃   ┃                   ┃   ┃
┣━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ┫
┃             * ┃               ┃           ┃   ┃           ┃
┣   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ╋━━━╋   ╋━━━┫
┃   ┃       ┃ *         ┃       ┃   ┃   ┃       ┃   ┃       ┃
┣━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ┫
┃   ┃ *   *   *     ┃   ┃       ┃   ┃   ┃               ┃   ┃
┣   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━┫
┃     * ┃           ┃       ┃       ┃       ┃       ┃   ┃   ┃
┣━━━╋   

#### c) Test de solve_bfs(start, stop)

In [16]:
laby = Maze(4,4, empty = True)
laby = laby.gen_exploration(15, 15)
solution = laby.solve_bfs((0, 0), (14, 14))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D ┃               ┃                                       ┃
┣   ╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ┫
┃ * ┃       ┃           ┃   ┃       ┃           ┃       ┃   ┃
┣   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━┫
┃ * ┃   ┃   ┃       ┃           ┃   ┃   ┃   ┃       ┃       ┃
┣   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋━━━╋   ┫
┃ * ┃           ┃       ┃       ┃           ┃               ┃
┣   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━┫
┃ * ┃               ┃   ┃   ┃ *   *   *   * ┃               ┃
┣   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ┫
┃ * ┃           ┃           ┃ * ┃       ┃ * ┃           ┃   ┃
┣   ╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ┫
┃ * ┃   ┃ *   * ┃ *   *   *   * ┃       ┃ * ┃           ┃   ┃
┣   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ┫
┃ *   *   * ┃ *   * ┃       ┃       ┃   ┃ *   *   *   * ┃   ┃
┣━━━╋━━━

#### d) Test de solve_rhr(start, stop)

In [5]:
laby = Maze(4,4, empty = True)
laby = laby.gen_fusion(15, 15)
solution = laby.solve_rhr((0, 0), (14, 14))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D   *     ┃ *   *     ┃       ┃       ┃                   ┃
┣━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ┫
┃   ┃ * ┃   ┃ * ┃ *   *   *   * ┃   ┃   ┃ *   * ┃           ┃
┣   ╋   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋━━━┫
┃   ┃ *   *   * ┃       ┃ *   * ┃       ┃ * ┃ *     ┃       ┃
┣   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━┫
┃   ┃       ┃   ┃         *             ┃ * ┃ *     ┃       ┃
┣   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ┫
┃       ┃           ┃     *   *   *   *   * ┃ *         ┃   ┃
┣━━━╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ┫
┃   ┃       ┃       ┃               ┃     *   *             ┃
┣   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━┫
┃                   ┃   ┃   ┃   ┃         * ┃   ┃   ┃       ┃
┣   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ┫
┃   ┃   ┃           ┃   ┃   ┃   ┃   ┃   ┃ *     ┃   ┃       ┃
┣   ╋━━━

## V) Évaluation

#### a) Test de distance_geo(c1, c2)

In [15]:
laby = Maze(4,4, empty = True)
laby = laby.gen_fusion(6, 6)
print(laby)
laby.distance_geo((1,1),(2,1))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃           ┃           ┃
┣   ╋   ╋━━━╋   ╋━━━╋━━━┫
┃   ┃   ┃               ┃
┣   ╋━━━╋━━━╋   ╋━━━╋━━━┫
┃           ┃           ┃
┣━━━╋   ╋   ╋━━━╋━━━╋   ┫
┃   ┃   ┃               ┃
┣   ╋━━━╋   ╋   ╋━━━╋━━━┫
┃           ┃       ┃   ┃
┣━━━╋   ╋━━━╋━━━╋   ╋   ┫
┃           ┃           ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┻━━━┛



5

#### b) Test de distance_man(c1, c2)

In [27]:
laby = Maze(4,4, empty = True)
laby = laby.gen_fusion(6, 6)
print(laby)
laby.distance_man((1,1),(4,1))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃               ┃   ┃   ┃
┣   ╋━━━╋━━━╋   ╋   ╋   ┫
┃       ┃               ┃
┣   ╋   ╋   ╋   ╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋   ╋━━━╋   ╋   ┫
┃                       ┃
┣   ╋━━━╋━━━╋   ╋━━━╋   ┫
┃       ┃   ┃   ┃   ┃   ┃
┣   ╋   ╋   ╋   ╋   ╋   ┫
┃   ┃   ┃       ┃       ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┻━━━┛



3